In [1]:
# %matplotlib inline
import plotly.offline as go_offline
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Функция, которая на вход получает DataFrame, а возвращает DataFrame с заполненным столбцом generation
# В столбце generation в каждой строке отображается название поколения, к которому принадлежит ребенок
# Каждое поколение - это 20 лет истории (подробнее: https://ru.wikipedia.org/wiki/Поколение)

def change_value_gen(df):
    generations = ['the greatest', 'the silent', 'baby boom',
                   'generation x', 'generation y', 'generation z']
    current_gen = len(generations) - 1

    for year in range(2020, 1919, -20):
        df['generation'] = np.where(df['year of birth'] < year,
                                      generations[current_gen], df['generation'])
        current_gen -= 1

    del df['year of birth']
    return df

In [3]:
# Данные о новорожденных содержат: штат, в котором родился ребенок (state), пол ребенка (sex),
# год рождения (year of birth), имя (name), и число новорожденных с этим именем (number)

data = pd.read_csv('states_data.csv')

# Создаем столбец поколение (generation) и наполняем его значением 'generation'
data['generation'] = 'generation'
data.head()

,state,sex,year of birth,name,number,generation
0,IN,F,1910,Mary,619,generation
1,IN,F,1910,Helen,324,generation
2,IN,F,1910,Ruth,238,generation
3,IN,F,1910,Dorothy,215,generation
4,IN,F,1910,Mildred,200,generation


In [4]:
# Вызов функции для переопределения значений в столбце поколение (generation)

data = change_value_gen(data)

In [5]:
# Группируем значения по имени, полу, поколению и штату
# В столбце number рассчитываем сумму

data = data.groupby(['name', 'sex', 'generation', 'state'], as_index=False, sort=False).agg({'number': 'sum'})
data.head()

,name,sex,generation,state,number
0,Mary,F,the greatest,IN,15575
1,Helen,F,the greatest,IN,7935
2,Ruth,F,the greatest,IN,5327
3,Dorothy,F,the greatest,IN,6306
4,Mildred,F,the greatest,IN,4243


In [6]:
# Сортируем значения по чилу новорожденных (number) в порядке убывания
# А сортировка по штатам и поколению - в порядке возрастания 

data = data.sort_values(by='number', ascending=False)
data = data.sort_values(by=['state', 'generation', 'sex'])
data.set_index(['state', 'generation', 'sex'], inplace=True)
data.head()

name  number
state generation sex                  
AK    baby boom  F        Mary    1349
                 F       Linda    1008
                 F       Susan     811
                 F    Patricia     761
                 F       Debra     648

In [7]:
# Удаляем все значения из DF, кроме первого, где число новорожденных максимально
# Сбрасываем индексы

data = data.loc[~data.index.duplicated(keep='first')].reset_index()

In [8]:
# Для построения интерактивной карты воспользуемся лишь часть данных
# В примере рассмотрим построение интерактивной карты на поколении baby boom, пол новорожденных - мужской

data = data[(data['generation'] == 'baby boom') & 
           (data['sex'] == 'M')]

In [9]:
# На основе данных строим интерактивную карту, 
# где в каждом штате будет отображено наиболее популярное имя ребенка
# поколения Baby Boom (карта-файл 'BB_M_map.html')

fig = px.choropleth(data, locations='state', color='name',
                    color_discrete_sequence=px.colors.qualitative.Antique,
                    locationmode = 'USA-states',
                    scope='usa',
                    hover_data=['name', 'number']
                   )
fig.update_layout(
    title_text = 'Popular names of "Baby Boom" generation (Male)',
)
# go_offline.plot(fig,filename='BB_M_map.html',validate=True, auto_open=False)

'BB_M_map.html'

In [10]:
# В результате наиболее популярные мужские имена для новорожденных в 1940-1959 годах - Robert и James
# 832 808 новорожденных в 1940-1959 носят имя "Роберт" 
data.groupby('name', as_index=False).number.sum()

,name,number
0,David,76871
1,James,832808
2,John,178758
3,Michael,4139
4,Robert,734252


In [11]:
# Только в 5 штатах в то время новорожденных мальчиков называли John
# В 4 штатах - David
# А на Гавайях самое популярное мужское имя - Michael

data.groupby('name', as_index=False).state.count()

,name,state
0,David,4
1,James,20
2,John,5
3,Michael,1
4,Robert,21
